# Build metadata GHS-composite-S2 R2020A

This section covers mirroring locally the UK section of the cloud-free composite Sentinel-2 mosaic created by the European Commission. Official website is over at:

> https://ghsl.jrc.ec.europa.eu/ghs_s2composite.php

And paper for the dataset is:

> Corbane, C., Politis, P., Kempeneers, P., Simonetti, D., Soille, P., Burger, A., ... & Kemper, T. (2020). [A global cloud free pixel-based image composite from Sentinel-2 data](https://www.sciencedirect.com/science/article/pii/S2352340920306314). *Data in Brief*, 105737.

In [ ]:
try:
    from download import download
except:
    !pip install download
    from download import download
import sys
sys.path.insert(0, "../")
import utils
import rasterio
import rioxarray
import pandas
import geopandas
from shapely.geometry import box
from datetime import datetime

## Local/remote paths



The base URL for the dataset's FTP is:

In [2]:
base_url = ("http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/"\
            "GHSL/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A/"\
            "GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A_UTM_10/V1-0/"
           )
base_url

'http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A_UTM_10/V1-0/'

## Tile listing

According to Corbane et al. (2020), the dataset is split on UTM tiles, each of them further subdivided on small GeoTIFF files. 

The tiles required to cover Great Britain are:

In [3]:
gb_utm_tiles = ["30U", "31U", "29V", "30V"]

And all of the UTM tiles covered by the dataset can be accessed from the official FTP site:

In [4]:
%time utm_tiles = pandas.read_html(base_url, skiprows=2)[0].iloc[:-1, :]

CPU times: user 171 ms, sys: 11.2 ms, total: 182 ms
Wall time: 1min 13s


This gives us the full list of tables available. However, tiles on the `1` do not appear on the [official page map](https://ghsl.jrc.ec.europa.eu/download.php?ds=compositeS2), so we will remove them. The final object is a list we can easily iterate over:

In [5]:
utm_tile_ids = [i for i in utm_tiles["Parent Directory"] \
                if not (i[0]=="1" and len(i)==3)]

## Virtual rasters

Each UTM tile has a `.vrt` file, a XML file that lists each component. For example, for the 30U tile, it is available at:

In [10]:
p = f"{base_url}{utm_tile_ids[0]}{utm_tile_ids[0].strip('/')}_UTM.vrt"
p

'http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A_UTM_10/V1-0/2K/2K_UTM.vrt'

Since `rasterio` supports `.vrt` files, we can inspect it:

In [11]:
%time tile30U = rasterio.open(p)

CPU times: user 3.97 ms, sys: 3.96 ms, total: 7.93 ms
Wall time: 613 ms


You can get a full list of the components of the virtual raster with `files`:

In [12]:
tile30U.files

['/vsicurl/http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A_UTM_10/V1-0/2K/2K_UTM.vrt',
 '/vsicurl/http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A_UTM_10/V1-0/2K/2K_UTM.vrt.ovr',
 '/vsicurl/http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A_UTM_10/V1-0/2K/2K_UTM.vrt.ovr.ovr',
 '/vsicurl/http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A_UTM_10/V1-0/2K/2K_UTM.vrt.ovr.ovr.ovr',
 '/vsicurl/http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A/GHS_composite_S2_L1C_2017-2018_GLOBE_R2020A_UTM_10/V1-0/2K/2K_UTM.vrt.ovr.ovr.ovr.ovr',
 '/vsicurl/http://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/G

## Build index GeoJSON

Here we build a GeoJSON file with every single GeoTIFF file that makes up the mosaic of all tiles. The strategy is:

1. Loop over UTM tiles
1. For each UTM, read `.vrt` and pull out GeoTIFF components
1. For each GeoTIFF, extract:
    - URL
    - CRS
    - UTM tile
    - Corners
    - Polygon with corners
1. Build `GeoDataFrame` for each UTM and convert to `EPSG:4326`
1. Concatenate all tables into a single one
1. Write out

Much of the heavy-lifting for each tile is done on `utils.parse_tile_meta`, the rest of is done in the following loop:

In [13]:
%%time
outfile = "GHS-composite-S2.geojson"
! rm -rf $outfile
meta = []
for tile in utm_tile_ids:
    tile_meta = utils.parse_tile_meta(tile)
    meta.append(tile_meta.to_crs(epsg=4326))
meta = pandas.concat(meta).reset_index(drop=True)
meta.to_file(outfile, driver="GeoJSON")

29/10/2020 14:35:41 | Working on tile 2K
29/10/2020 14:35:42 | 10 files to process
	29/10/2020 14:35:42 | Working on file S2_percentile_30_UTM_721-0000069888-0000000000.tif
	29/10/2020 14:35:43 | Working on file S2_percentile_30_UTM_721-0000046592-0000046592.tif
	29/10/2020 14:35:43 | Working on file S2_percentile_30_UTM_721-0000046592-0000023296.tif
	29/10/2020 14:35:44 | Working on file S2_percentile_30_UTM_721-0000046592-0000000000.tif
	29/10/2020 14:35:44 | Working on file S2_percentile_30_UTM_721-0000023296-0000046592.tif
	29/10/2020 14:35:45 | Working on file S2_percentile_30_UTM_721-0000023296-0000023296.tif
	29/10/2020 14:35:46 | Working on file S2_percentile_30_UTM_721-0000023296-0000000000.tif
	29/10/2020 14:35:47 | Working on file S2_percentile_30_UTM_721-0000000000-0000046592.tif
	29/10/2020 14:35:47 | Working on file S2_percentile_30_UTM_721-0000000000-0000023296.tif
	29/10/2020 14:35:48 | Working on file S2_percentile_30_UTM_721-0000000000-0000000000.tif
29/10/2020 14:35: